## Decorator

進階的程式碼通常會有裝飾器，為了要邁向更高的程式設計的境界，有必要了解本節的裝飾器

- 函式是物件，可被賦值給變數，可作為參數傳遞，也可作為其他函數的回傳值
- 函式裡面可以再定義函式，子函式可以記住父函式的狀態，就稱為閉包

### 1.閉包介紹

閉包範例
- 函式裡定義函式
- 裡面函式用了外面函式的變量
- 外面函式返回裡面函式的名稱

In [4]:
def fun1(num1):
    print('Step1')
    x = 5
    def fun2(num2):
        print('Step2')
        print(x*num2 + num1)
    return fun2

In [6]:
f = fun1(100) # f 指向 fun2
print('Step3')
f(5) # 呼叫 fun2

Step1
Step3
Step2
125


外面函式執行結束後，裡面資料仍被紀錄；即使外面被刪除，傳入閉包的資料仍然留著！

In [8]:
f = fun1(100) # f 指向 fun2
f(5)
print('Step3')
del fun1
f(5)

Step1
Step2
125
Step3
Step2
125


### 2.裝飾器介紹

裝飾器是在修飾另外一個函式，可以增加額外的程式碼在之前或之後於被包裝的函式

想像裝飾器是燈罩，加上燈罩之後，可以修改電燈照明範圍；你也可以改燈罩的顏色，讓他有不同顏色的輸出

但電燈的功能本身沒有改變，只是被燈罩修改了照明範圍或顏色

裝飾器如同上面的舉例，可以隨時加上去很方便！

裝飾器有以下優點：

- 擴充容易
- 重複利用

In [38]:
def say_hello():
    return 'Hello'

In [39]:
say_hello()

'Hello'

In [51]:
def add_something(func):
    print('something')
    return func

In [52]:
new_func = add_something(say_hello)
new_func()

something


'Hello'

我們可以用 python 的語法糖來改寫

In [56]:
@add_something
def say_hello():
    return 'Hello'
say_hello()

something


'Hello'

製作一個小寫裝飾器

In [60]:
def lowercase(func):
    def wrapper_func(*args):
        print("原函式執行前")
        original_result = func(*args)
        print("original_result:", original_result)
        print("原函式執行後")
        modified_result = original_result.lower()
        print("modified_result:", modified_result)
        return modified_result
    return wrapper_func

@lowercase
def say_hello():
    return 'Hello'

In [61]:
say_hello()

原函式執行前
original_result: Hello
原函式執行後
modified_result: hello


'hello'

In [62]:
def uppercase(func):
    def wrapper_func(*args):
        original_result = func(*args)
        modified_result = original_result.upper()
        return modified_result
    return wrapper_func

@uppercase
def say_hello():
    return 'Hello'

In [63]:
say_hello()

'HELLO'

#### 講師提醒
\*args : 收集所有位置型參數，以 tuple 存在 args

\**kwargs: 收集所有關鍵字參數，以 dict 存在 kwargs

In [69]:
import datetime

def timer(func):
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()
        result = func(*args, **kwargs)
        end = datetime.datetime.now()
        print(f"執行時間：{ end - start } 秒")
        return result
    return wrapper

In [71]:
@timer
def sleep():
    time.sleep(1)

In [72]:
sleep()

執行時間：0:00:01.001548 秒


### 3.functions.wraps

撰寫裝飾器是把一個函式置換成另一個，原本的函式名稱和docstring等等都會不見！！

我們可以利用這個套件來避免這個問題

In [75]:
def sleep(n):
    '''
    想睡覺的程式
    n: 睡覺秒數
    '''
    time.sleep(n)

In [76]:
sleep.__doc__

'\n    想睡覺的程式\n    n: 睡覺秒數\n    '

In [77]:
sleep.__name__

'sleep'

In [79]:
import datetime

def timer(func):
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()
        result = func(*args, **kwargs)
        end = datetime.datetime.now()
        print(f"執行時間：{ end - start } 秒")
        return result
    return wrapper

@timer
def sleep(n):
    '''
    想睡覺的程式
    n: 睡覺秒數
    '''
    time.sleep(n)

In [80]:
sleep.__doc__

In [81]:
sleep.__name__

'wrapper'

In [93]:
import datetime
import functools

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()
        result = func(*args, **kwargs)
        end = datetime.datetime.now()
        print(f"執行時間：{ end - start } 秒")
        return result
    return wrapper

@timer
def sleep(n):
    '''
    想睡覺的程式
    n: 睡覺秒數
    '''
    time.sleep(n)

In [83]:
sleep.__doc__

'\n    想睡覺的程式\n    n: 睡覺秒數\n    '

In [84]:
sleep.__name__

'sleep'

### 4.輸入參數

如果想讓裝飾器傳入參數，我們可以再用另一個函式包住修飾器

In [97]:
import datetime
import functools

def timer_repeat(n):
    def timer(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start = datetime.datetime.now()
            result = func(*args, **kwargs)
            end = datetime.datetime.now()
            for _ in range(n):
                print(f"執行時間：{ end - start } 秒")
            return result
        return wrapper
    return timer

@timer_repeat(5)
def sleep(t):
    '''
    想睡覺的程式
    n: 睡覺秒數
    '''
    time.sleep(t)

In [98]:
sleep(1)

執行時間：0:00:01.004990 秒
執行時間：0:00:01.004990 秒
執行時間：0:00:01.004990 秒
執行時間：0:00:01.004990 秒
執行時間：0:00:01.004990 秒
